# Dialogue Summarization EDA

이 노트북은 대회용 `train/dev/test` 대화 요약 데이터를 탐색하고,
모델링/전처리/프롬프트 설계를 위한 인사이트를 얻기 위한 EDA 계획 및 실행을 담습니다.

## 1. EDA 전체 계획

1. **데이터 구조 파악**  
   - `train/dev/test` 행 개수, 컬럼(`fname, dialogue, summary, topic`) 구조 확인  
   - topic 분포 (train/dev) 비교, test에 topic 유무 확인

2. **대화/요약 길이 통계**  
   - 문자/토큰 기준 길이 분포 (평균, 중앙값, 상/하위 quantile)  
   - encoder_max_len=1024, decoder_max_len=80 기준으로 잘리는 비율 추정  
   - 발화(turn) 개수 분포 (#Person1/#Person2 등장 회수)

3. **토픽/스타일 분석**  
   - topic별 평균 대화 길이 / 요약 길이 비교  
   - 대표적인 대화-요약 페어 몇 개 샘플링해서, 요약 스타일(존댓말/반말, 요약의 추상화/추출 비율) 파악  
   - 요약에서 자주 등장하는 키워드/표현 조사 (간단한 빈도/WordCloud 수준)

4. **train/dev 데이터 분포 차이 체크**  
   - 길이 분포, topic 분포, 자주 등장하는 단어 등 비교  
   - 분포가 비슷한지, dev가 특정 topic에 편향되어 있는지 확인

5. **모델/프롬프트 설계에 직접 연결되는 분석**  
   - 긴 대화에서 앞/뒤 어느 부분이 요약에 더 많이 반영되는지 (간단한 n-gram overlap 위치 분석)  
   - 스타일 프롬프트 후보 문구를 몇 개 정의하고, 요약 스타일과 매칭되는지 눈으로 검증  
   - KoBART/T5 토크나이저 기준 토큰 길이 분포 (추후 필요 시)

6. **제출 파일/파이프라인 sanity check**  
   - `prediction/*.csv`에서 `fname` 정렬/중복 여부, summary 공백/길이 확인  
   - 극단적으로 짧거나 긴 요약 몇 개 샘플링하여 품질/스타일 확인

In [15]:
# 데이터 로딩 셀입니다.
# - notebooks/ 폴더에서 실행해도 항상 프로젝트 루트의 data/를 바라보도록 경로를 처리합니다.
import pandas as pd
from pathlib import Path

# 이 노트북이 있는 위치 기준으로 프로젝트 루트 추정
NOTEBOOK_DIR = Path().resolve()
ROOT_DIR = NOTEBOOK_DIR.parent  # nlp_contest/
DATA_DIR = ROOT_DIR / "data"

TRAIN_PATH = DATA_DIR / "train.csv"
DEV_PATH = DATA_DIR / "dev.csv"
TEST_PATH = DATA_DIR / "test.csv"

missing = [p for p in [TRAIN_PATH, DEV_PATH, TEST_PATH] if not p.exists()]
if missing:
    missing_str = ", ".join(str(p) for p in missing)
    raise FileNotFoundError(
        f"다음 파일을 찾을 수 없습니다: {missing_str}\n"
        "- 프로젝트 루트(nlp_contest) 아래 data/에 csv를 넣은 뒤 이 셀을 다시 실행하세요."
    )

train_df = pd.read_csv(TRAIN_PATH)
dev_df = pd.read_csv(DEV_PATH)
test_df = pd.read_csv(TEST_PATH)

train_df.head(), dev_df.head(), test_df.head()


(     fname                                           dialogue  \
 0  train_0  #Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...   
 1  train_1  #Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...   
 2  train_2  #Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 어떤 ...   
 3  train_3  #Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...   
 4  train_4  #Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...   
 
                                              summary      topic  
 0  Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...       건강검진  
 1  Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...      백신 접종  
 2  #Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...      열쇠 분실  
 3  #Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...  여자친구와의 결혼  
 4  Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...       춤 제안  ,
    fname                                           dialogue  \
 0  dev_0  #Person1#: 안녕하세요, 오늘 기분이 어떠세요?\n#Person2#: 요즘 ...   
 1  dev_1  #Person1#: 야 Jimmy, 오늘 좀 이따 운동하러 가자.\n#Person2...   
 2  dev

## 2. 데이터 구조 및 기본 통계

- 각 split의 행 개수, 컬럼, 결측치, topic 유무 등 기본 정보 확인

In [16]:
print("Train shape:", train_df.shape)
print("Dev shape:", dev_df.shape)
print("Test shape:", test_df.shape)

print("\nTrain columns:", train_df.columns.tolist())
print("Dev columns:", dev_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())

print("\nTrain topic value_counts:\n", train_df.get("topic", pd.Series()).value_counts().head())
print("\nDev topic value_counts:\n", dev_df.get("topic", pd.Series()).value_counts().head())

Train shape: (12457, 4)
Dev shape: (499, 4)
Test shape: (499, 2)

Train columns: ['fname', 'dialogue', 'summary', 'topic']
Dev columns: ['fname', 'dialogue', 'summary', 'topic']
Test columns: ['fname', 'dialogue']

Train topic value_counts:
 topic
음식 주문     130
취업 면접     109
길 안내       66
호텔 체크인     40
아파트 임대     30
Name: count, dtype: int64

Dev topic value_counts:
 topic
호텔 방 예약    5
길 안내       4
취업 면접      4
음식 주문      4
신발 구매      2
Name: count, dtype: int64


## 3. 길이 분포 분석 (문자 기준)

- dialogue 길이, summary 길이의 기본 통계를 보고, encoder/decoder max length 설정이 적절한지 감각을 잡습니다.

In [17]:
for split_name, df in [("train", train_df), ("dev", dev_df)]:
    df["dialogue_len_char"] = df["dialogue"].astype(str).str.len()
    df["summary_len_char"] = df.get("summary", "").astype(str).str.len()

    print(f"==== {split_name.upper()} ====")
    print("dialogue_len_char describe:\n", df["dialogue_len_char"].describe())
    print("summary_len_char describe:\n", df["summary_len_char"].describe())
    print()

==== TRAIN ====
dialogue_len_char describe:
 count    12457.000000
mean       406.083487
std        197.566083
min         84.000000
25%        280.000000
50%        369.000000
75%        500.000000
max       2165.000000
Name: dialogue_len_char, dtype: float64
summary_len_char describe:
 count    12457.000000
mean        85.789436
std         33.811948
min         13.000000
25%         61.000000
50%         80.000000
75%        104.000000
max        376.000000
Name: summary_len_char, dtype: float64

==== DEV ====
dialogue_len_char describe:
 count     499.000000
mean      400.054108
std       186.163807
min       114.000000
25%       273.000000
50%       367.000000
75%       487.000000
max      1269.000000
Name: dialogue_len_char, dtype: float64
summary_len_char describe:
 count    499.000000
mean      81.206413
std       32.577548
min       29.000000
25%       58.000000
50%       74.000000
75%       96.000000
max      283.000000
Name: summary_len_char, dtype: float64



## 4. 발화(turn) 수 분포

- `#Person1#`, `#Person2#` 패턴을 이용해 대화 turn 수 분포를 대략적으로 확인합니다.

In [18]:
import re

def count_turns(text: str) -> int:
    return len(re.findall(r"#Person[0-9]+#", str(text)))

train_df["num_turns"] = train_df["dialogue"].apply(count_turns)
dev_df["num_turns"] = dev_df["dialogue"].apply(count_turns)

print("Train num_turns describe:\n", train_df["num_turns"].describe())
print("Dev num_turns describe:\n", dev_df["num_turns"].describe())

Train num_turns describe:
 count    12457.000000
mean         9.491451
std          4.146670
min          2.000000
25%          7.000000
50%          9.000000
75%         12.000000
max         59.000000
Name: num_turns, dtype: float64
Dev num_turns describe:
 count    499.000000
mean       9.398798
std        4.010438
min        2.000000
25%        6.000000
50%        9.000000
75%       12.000000
max       29.000000
Name: num_turns, dtype: float64


## 5. topic별 길이/스타일 차이 (추후 확장)

- topic이 존재한다면, topic별로 길이/turn 수/요약 길이 차이를 비교합니다.
- 필요 시 시각화(히스토그램, 박스플롯)를 추가할 수 있습니다.

In [19]:
# topic별로 대화 길이 / 요약 길이 / 턴 수 통계를 보는 셀입니다.
# pandas 최신 버전에서는 여러 컬럼을 선택할 때 리스트로 넘겨야 하므로,
# ["col1", "col2", ...] 형태로 명시적으로 지정합니다.
if "topic" in train_df.columns:
    cols = ["dialogue_len_char", "summary_len_char", "num_turns"]
    topic_stats = train_df.groupby("topic")[cols].describe()
    topic_stats.head()

## 6. 샘플 대화-요약 페어 확인

- 무작위로 몇 개를 뽑아서, 요약 스타일과 프롬프트 설계 방향을 눈으로 확인합니다.

In [20]:
SAMPLE_N = 3
sample_rows = train_df.sample(SAMPLE_N, random_state=42)
for i, row in sample_rows.iterrows():
    print("==== SAMPLE ====\n")
    print("fname:", row.get("fname"))
    print("topic:", row.get("topic"))
    print("[DIALOGUE]\n", row["dialogue"][:1000])
    print("\n[SUMMARY]\n", row["summary"])
    print("\n\n")

==== SAMPLE ====

fname: train_396
topic: 저녁 초대
[DIALOGUE]
 #Person1#: 안녕하세요, 잭 있나요?
#Person2#: 전데요.
#Person1#: 잭! 나 로즈야.
#Person2#: 안녕, 로즈. 어떻게 지내?
#Person1#: 잘 지내, 고마워. 이번 주 토요일 저녁에 친구들 몇 명 초대했어. 너도 같이 올 수 있는지 궁금해서.
#Person2#: 좋네. 몇 시에 가면 될까?
#Person1#: 여섯 시 괜찮아?

[SUMMARY]
 로즈는 잭에게 전화하여 이번 주 토요일 저녁 식사에 초대한다.



==== SAMPLE ====

fname: train_247
topic: 면접 준비
[DIALOGUE]
 #Person1#: 저기요, 면접 보러 갈 때 뭐 입어야 할까요?
#Person2#: 정장에 넥타이를 매는 게 좋을 것 같아요.
#Person1#: 면접 중에 긴장할까 봐 걱정이에요.
#Person2#: 걱정 마세요. 그냥 최선을 다해서 자신을 잘 표현하세요.

[SUMMARY]
 #Person2#는 #Person1#에게 면접에서 정장과 넥타이를 착용하고 자신을 잘 표현하라고 조언합니다.



==== SAMPLE ====

fname: train_9260
topic: 사업 전략 인터뷰
[DIALOGUE]
 #Person1#: 존, 동기부여에 대해 몇 가지 질문이 있어요. 당신이 지역 사람들과 함께 사업을 시작한 이유가 뭔가요?
#Person2#: 음, 저는 항상 지역 산업을 돕기 위해 지역 사람들을 고용하려고 했어요. 하지만 이 지역은 스페인의 일부 지방처럼 실업률이 낮지 않아서, 지역 외부 사람들도 고용해야 해요.
#Person1#: 관리 스타일은 어떠세요? 존, 엄격한 관리자이신가요?
#Person2#: 아니요, 그렇게 생각하지 않아요. 저는 강한 성격을 가졌고, 관리자로서도 강한 편이긴 하지만, 사람들을 해고해야 할 때는 다섯 번, 열 번 더 기회를 주곤 해요.
#Person1#: 앞으로의 계

## 7. KoBART 토크나이저 기준 토큰 길이 분포

- 이 셀에서는 **KoBART 토크나이저**로 대화문을 토크나이즈했을 때 토큰 길이 분포를 봅니다.
- encoder_max_len=1024 설정이 얼마나 여유 있는지, 잘리는 샘플 비율이 어느 정도인지 확인하는 목적입니다.

In [21]:
# KoBART 토크나이저를 불러와서, 대화문의 토큰 길이 분포를 확인하는 셀입니다.
# - encoder_max_len=1024 설정으로 충분한지 확인하기 위함입니다.
from transformers import PreTrainedTokenizerFast

kobart_tok = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v1")

def token_len(text: str) -> int:
    return len(kobart_tok.encode(str(text), add_special_tokens=True))

train_df["dialogue_len_tok"] = train_df["dialogue"].apply(token_len)
dev_df["dialogue_len_tok"] = dev_df["dialogue"].apply(token_len)

print("[TRAIN] dialogue_len_tok describe:\n", train_df["dialogue_len_tok"].describe())
print("[DEV] dialogue_len_tok describe:\n", dev_df["dialogue_len_tok"].describe())
print("\n[TRAIN] encoder_max_len=1024 초과 비율:", (train_df["dialogue_len_tok"] > 1024).mean())
print("[DEV] encoder_max_len=1024 초과 비율:", (dev_df["dialogue_len_tok"] > 1024).mean())

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


[TRAIN] dialogue_len_tok describe:
 count    12457.000000
mean       199.561532
std         93.123736
min         42.000000
25%        141.000000
50%        183.000000
75%        245.000000
max       1079.000000
Name: dialogue_len_tok, dtype: float64
[DEV] dialogue_len_tok describe:
 count    499.000000
mean     196.547094
std       87.909245
min       54.000000
25%      139.000000
50%      179.000000
75%      242.500000
max      670.000000
Name: dialogue_len_tok, dtype: float64

[TRAIN] encoder_max_len=1024 초과 비율: 0.00024082844986754435
[DEV] encoder_max_len=1024 초과 비율: 0.0


## 8. 요약 길이 vs 대화 길이 상관 관계

- 이 셀에서는 **대화 길이(문자 기준)**와 **요약 길이(문자 기준)**의 상관 관계를 계산합니다.
- "대화가 길어질수록 요약도 길어지는지" 대략적인 경향을 보는 것이 목적입니다.

In [22]:
# 대화 길이(문자)와 요약 길이(문자)의 상관 관계를 계산하는 셀입니다.
# - scatter plot까지는 아니지만, 상관 계수로 대략적인 경향을 봅니다.
corr = train_df[["dialogue_len_char", "summary_len_char"]].corr()
print("[TRAIN] dialogue_len_char vs summary_len_char 상관 계수:\n", corr)

[TRAIN] dialogue_len_char vs summary_len_char 상관 계수:
                    dialogue_len_char  summary_len_char
dialogue_len_char           1.000000          0.716915
summary_len_char            0.716915          1.000000


## 9. 요약 스타일 키워드 빈도

- 이 셀에서는 요약문에서 자주 등장하는 단어를 세어 봅니다.
- "환자", "고객", "상담", "예약" 등 어떤 표현이 많이 나오는지 보고,
  요약 스타일(예: 의료/상담/일상 대화 비율)을 감각적으로 파악하는 것이 목적입니다.

In [23]:
# 요약문에서 자주 등장하는 단어의 빈도를 간단히 계산하는 셀입니다.
# - 형태소 분석까지는 하지 않고, 한글/영문/숫자 토큰 기준으로 rough하게 봅니다.
from collections import Counter
import re

def tokenize_korean(text: str):
    return re.findall(r"[가-힣A-Za-z0-9]+", str(text))

words = []
sample_summaries = train_df["summary"].dropna().sample(min(1000, len(train_df)), random_state=42)
for s in sample_summaries:
    words.extend(tokenize_korean(s))

counter = Counter(words)
print("[요약문 상위 50개 토큰 빈도]")
for w, c in counter.most_common(50):
    print(w, c)

[요약문 상위 50개 토큰 빈도]
Person1 1099
Person2 993
는 625
은 539
에게 353
대해 212
이 154
의 138
과 124
가 117
설명합니다 95
합니다 93
위해 90
수 82
것을 78
있습니다 61
말합니다 56
함께 55
더 54
있다고 53
있으며 50
요청합니다 46
한다 45
Mr 45
대한 42
이야기합니다 42
있는 40
생각합니다 40
그 37
그들은 36
하고 35
자신의 32
안내합니다 32
제안합니다 31
있다 31
싶어 30
후 30
두 28
것이라고 28
말한다 27
찾고 27
결국 27
이를 26
와 26
큰 23
새 23
것이 23
그리고 23
하며 22
설명한다 22


## 10. train vs dev 길이 분포 비교

- 이 셀에서는 train/dev 간에 **대화 길이/요약 길이 평균**이 얼마나 다른지 비교합니다.
- train과 dev 분포가 크게 다르면, dev 성능이 실제 test 분포와도 다를 수 있으므로,
  분포 차이를 확인하는 것이 목적입니다.

In [24]:
# train과 dev의 길이 분포(평균 기준)를 비교하는 셀입니다.
print("[TRAIN] dialogue_len_char mean:", train_df["dialogue_len_char"].mean())
print("[DEV]   dialogue_len_char mean:", dev_df["dialogue_len_char"].mean())
print("[TRAIN] summary_len_char mean:", train_df["summary_len_char"].mean())
print("[DEV]   summary_len_char mean:", dev_df["summary_len_char"].mean())

[TRAIN] dialogue_len_char mean: 406.0834871959541
[DEV]   dialogue_len_char mean: 400.0541082164329
[TRAIN] summary_len_char mean: 85.78943565866581
[DEV]   summary_len_char mean: 81.2064128256513


## 11. prediction sanity check

- 이 셀에서는 현재까지 생성한 **제출용 CSV**를 간단히 검증합니다.
- 예: `fname` 중복 여부, summary가 비어 있는지, 길이 분포가 너무 극단적이지 않은지 확인합니다.
- 아래 파일명은 예시이며, 실제로 확인하고 싶은 최신 CSV 경로로 수정해서 사용하면 됩니다.

In [25]:
# 생성된 prediction CSV가 기본 형식을 잘 따르는지 확인하는 셀입니다.
# - fname 중복/누락, summary 공백 비율, 요약 길이 분포 등을 간단히 체크합니다.

PRED_PATH = "prediction/2511292249_kobart-base-style_prompt_bs8.csv"  # 필요에 따라 최신 파일로 변경

try:
    pred_df = pd.read_csv(PRED_PATH)
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {PRED_PATH}")
else:
    print(pred_df.head())
    print("rows:", len(pred_df))
    print("fname unique:", pred_df["fname"].nunique())
    empty_ratio = (pred_df["summary"].astype(str).str.strip() == "").mean()
    print("empty summary 비율:", empty_ratio)
    print("summary 길이 통계:\n", pred_df["summary"].astype(str).str.len().describe())

파일을 찾을 수 없습니다: prediction/2511292249_kobart-base-style_prompt_bs8.csv
